In [25]:
1.28+2.374+0.866+0.52

5.039999999999999

In [ ]:
import pdb

# For checking progress
from tqdm import tqdm

# For loading data
import pandas as pd

# For tokenizaton
import nltk
from nltk import word_tokenize, sent_tokenize
nltk.download('punkt')

# For building n-gram model
from collections import Counter, namedtuple
import numpy as np

# For pos tagging
nltk.download('averaged_perceptron_tagger')
import re

def get_corpus():
  """ Reads and formats the corpus.

  Returns:
    corpus (list[str]):
      A list of sentences in the corpus.
  """
  df = pd.read_csv('https://raw.githubusercontent.com/yunzhusong/NLP109/main/lab1_data.csv')
  corpus = df.content.to_list()
  return corpus

def preprocess(documents):
  """ Preprocesses the corpus.
  
  Args:
    documents (list[str]):
      A list of sentences in the corpus.
  Returns:
    cleaned_documents (list[str]):
      A list of cleaned sentences in the corpus.
  """
  cleaned_documents = []
  punc = "\\【.*?】+|\\《.*?》+|\\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\\\，。=？、：「」『』￥……（）《》【】]"
  for doc in documents:
    # Tokenizes the sentence
    sents = sent_tokenize(doc)
    for sent in sents:
      #pdb.set_trace() # delete this line for the final version
 
      # Removes the punctuations, hint: recursively remove in character level
      sent = re.sub(punc,'',sent)

      # Lowers the case, 
      sent=sent.lower()

      cleaned_documents.append(sent)

  #print(cleaned_documents[:5])
  return cleaned_documents

# Compute word frequency
def get_vocab(documents):
  """ Gets the vocabulary from the corpus.
  
  Args:
    documents (list[str]):
      A list of sentences in the corpus
  Returns:
    vocabulary (collections.Counter)
  """
  vocabulary = Counter()

  for doc in tqdm(documents):
    tokens = word_tokenize(doc)
    vocabulary.update(tokens)

  return vocabulary



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Read data
raw_documents = get_corpus()

# Build vocabulary
vocab = get_vocab(raw_documents).most_common(10)
print('\n Before preprocessing:', vocab)

# Build vocabulary after preprocessing
documents = preprocess(raw_documents)
vocab = get_vocab(documents).most_common(10)
print('\n After preprocesing:', vocab)

100%|██████████| 100000/100000 [00:21<00:00, 4555.93it/s]



 Before preprocessing: [('.', 85947), ('the', 49772), (',', 39728), ('to', 34407), ('!', 33580), ('a', 28765), ('is', 26339), ('?', 24057), ('and', 22890), ('of', 22542)]


100%|██████████| 175323/175323 [00:18<00:00, 9264.45it/s]


 After preprocesing: [('the', 55985), ('to', 34928), ('a', 30055), ('you', 27036), ('is', 26952), ('and', 25375), ('of', 22989), ('that', 16030), ('it', 15936), ('i', 15848)]


gerGEWgrgre


IndexError: ignored

In [ ]:
doc=['gerGEWgr gre','gre/34$# gre r','gre4']
for d in doc:
  print(d)
  

gerGEWgr gre
gergewgr gre
gre/34$# gre r
gre/34$# gre r
gre4
gre4


In [ ]:
class Ngram_model(object):
  """ Ngram model implementation.

  Attributes:
    n (int):
      The number of grams to be considered.
    model (dict):
      The ngram model.
  """
  def __init__(self, documents, N=2):
    self.n = N
    self.model = self.get_ngram_model(documents)

  def get_ngram_model(self, documents):
    N = self.n
    ngram_model = dict()
    full_grams = list()
    grams = list()
    Word = namedtuple('Word', ['word', 'prob'])

    # for each sentence in documents
    for sent in documents:
      
      # Tokenizes to words
      sent = word_tokenize(sent)
      
      # Append (N-1) start tokens '<s>' and an end token '<\s>'
      temp1=[]
      for i in range(N-1):
        temp1.append('<s>')
      sent = temp1 + sent
      sent.append('<\s>')

      # Calculates numerator (construct list with full grams, i.e., N-grams)
      
      if (N==2):
        for i in range(len(sent)-1):
          temp2=(sent[i],sent[i+1])
          full_grams.append(temp2)
      elif (N==4):
        for i in range(len(sent)-3):
          temp2=(sent[i],sent[i+1],sent[i+2],sent[i+3])
          full_grams.append(temp2)
      
      

      # Calculate denominator (construct list with grams, i.e., (N-1)-grams)
      
      if (N==2):
        for i in range(len(sent)):
          temp3=(sent[i],)
          grams.append(temp3)
      elif (N==4):
        for i in range(len(sent)-2):
          temp3=(sent[i],sent[i+1],sent[i+2])
          grams.append(temp3)
      
    # Count the occurence frequency of each gram
    # Take 2-gram model as example:
    #   full_grams -> list[('a', 'gram'),('other', 'gram'), ...]
    #   grams -> list[('a'), ('other'), ('gram'), ...]
    #   full_gram_counter -> dict{('a', 'gram'):frequency_1, ('other','gram'):frequency_2, ...}
    #   gram_counter -> dict{('a'):frequency_1, ('gram'):frequency_2, ...}
    full_gram_counter = Counter(full_grams)
    gram_counter = Counter(grams)

    # Build model
    # Take 2-gram model as example:
    #   { '<s>': [tuple(word='i', prob=0.6), tuple(word='the', prob=0.2), ...],
    #   'i': [tuple(word='am', prob=0.7), tuple(word='want', prob=0.1), ...],
    #    ... }
    for key in full_gram_counter:
      word = ''.join(key[:N-1])

      if word not in ngram_model:
        ngram_model.update({word: set()})

      # next_word_prob -> float
      next_word_prob = full_gram_counter[key] / gram_counter[key[:N-1]]
      w = Word(key[-1], next_word_prob)
      ngram_model[word].add(w)

    # Sort the result by frequency
    for word, ng in ngram_model.items():
      ngram_model[word] = sorted(ng, key=lambda x: x.prob, reverse=True)

    return ngram_model


  def predict_sent(self, text=None, max_len=30):
    """ Predicts a sentence with the ngram model.

    Args:
      text (string or list[string])
    Returns:
      A prediction string.
    """

    N = self.n
    backup_tokens = ['<s>']*(N-1)
    if not text:
      tokens = backup_tokens
      output = []

    elif type(text)==str:
      tokens = backup_tokens + text.split(' ')
      tokens = tokens[-(N-1):]
      if not self.check_existence(tokens):
        return 
      output = tokens

    elif type(text) == list:
      tokens = backup_tokens + text
      tokens = tokens[-(N-1):]
      if not self.check_existence(tokens):
        return
      output = tokens

    else:
      print('[Error] the input text must be string or list of string')
      return

    for i in range(max_len):
      possible_words = list(self.model[''.join(tokens)])
      probs = [word.prob for word in possible_words]
      words = [word.word for word in possible_words]
      next_word = np.random.choice(words, 1, p=probs)[0]
      tokens = tokens[1:] + [next_word]

      if next_word == '<\\s>':
        break

      output.append(next_word)
    return ' '.join(output)

  def predict_next(self, text=None, top=5):
    """ Predicts next word with the ngram model.

    Args:
      text (string or list[string])

    Returns:
      possible_next_words (list[namedtuple]):
        A list of top few possible next words.
    """

    N = self.n
    backup_tokens = ['<s>']*(N-1)
    if not text:
      tokens = backup_tokens

    elif type(text)==str:
      tokens = backup_tokens + text.split(' ')
      tokens = tokens[-(N-1):]
      if not self.check_existence(tokens):
        return 

    elif type(text) == list:
      tokens = backup_tokens + text
      tokens = tokens[-(N-1):]
      if not self.check_existence(tokens):
        return
    else:
      print('[Error] the input text must be string or list of string')

    possible_next_words = self.model[''.join(tokens)][:top]
    possible_next_words = [(word.word, word.prob) for word in possible_next_words]

    return possible_next_words

  def check_existence(self, tokens):
    if not ''.join(tokens) in self.model.keys():
      print('[Error] the input text {} not in the vocabulary'.format(tokens))
      return False
    else:
      return True





In [ ]:
twogram = Ngram_model(documents, N=2)
fourgram = Ngram_model(documents, N=4)

In [ ]:
output = twogram.predict_next(text='<s>', top=5)
print('Next word predictions of two gram model:', output)

Next word predictions of two gram model: [('i', 0.05122545245061971), ('you', 0.03139918892558307), ('the', 0.030748960490066906), ('<\\s>', 0.030623477809528697), ('they', 0.018776772015080736)]


In [ ]:
output = twogram.predict_sent(max_len=30)
print('Generation results of two gram model:', output)
nltk.pos_tag(word_tokenize(output))

Generation results of two gram model: guyverhofstadt


[('guyverhofstadt', 'NN')]

In [ ]:
output = fourgram.predict_sent(max_len=30)
print('Generation results of four gram model: ', output)
nltk.pos_tag(word_tokenize(output))

Generation results of four gram model:  the us corporate lame stream media so afraid of upsetting minority groups amp ; ethnic minorities they concoct stories to deflect from the fact that india today group tolerating this


[('the', 'DT'),
 ('us', 'PRP'),
 ('corporate', 'JJ'),
 ('lame', 'NN'),
 ('stream', 'NN'),
 ('media', 'NNS'),
 ('so', 'RB'),
 ('afraid', 'JJ'),
 ('of', 'IN'),
 ('upsetting', 'JJ'),
 ('minority', 'NN'),
 ('groups', 'NNS'),
 ('amp', 'VBP'),
 (';', ':'),
 ('ethnic', 'JJ'),
 ('minorities', 'NNS'),
 ('they', 'PRP'),
 ('concoct', 'VBP'),
 ('stories', 'NNS'),
 ('to', 'TO'),
 ('deflect', 'VB'),
 ('from', 'IN'),
 ('the', 'DT'),
 ('fact', 'NN'),
 ('that', 'IN'),
 ('india', 'JJ'),
 ('today', 'NN'),
 ('group', 'NN'),
 ('tolerating', 'VBG'),
 ('this', 'DT')]